In [2]:
from typing import List
from roc_800.client import ROCPlusClient
from roc_800.models import HistorySegmentPointConfiguration
from tlp_models.tlp import TLPInstance
from tlp_models.point_types import PointTypes
import struct
roc_client = ROCPlusClient(
    ip='10.248.213.37',
    port=10001,
    roc_address=166,
    roc_group=100
)
tlp_def = TLPInstance(parameter=PointTypes.HISTORY_SEGMENT_0_POINT_CONFIGURATION.Parameters.PARAMETER_DESCRIPTION, logical_number=2)
async with roc_client.connect(init_history_def=False, init_io_def=True, init_user_opcode_def=True) as client:
    print(client.get_config_json())
    # config = await client.get_system_config()
    # print(config.model_dump_json(indent=2))

2025-01-09 14:59:10.398 | DEBUG    | roc_800.client:connect:87 - Opening connection within async context manager...
2025-01-09 14:59:10.402 | DEBUG    | tcp_client:open_connection:48 - Ensuring ROC Client connection.
2025-01-09 14:59:10.402 | DEBUG    | tcp_client:open_connection:51 - Stream does not exist. Opening connection...
2025-01-09 14:59:10.455 | DEBUG    | tcp_client:open_connection:59 - Stream connected successfully.
2025-01-09 14:59:10.455 | DEBUG    | roc_800.client:connect:90 - Connected successfully.
2025-01-09 14:59:10.456 | DEBUG    | roc_800.client:connect:92 - Reading I/O definition.
2025-01-09 14:59:10.456 | DEBUG    | roc_800.client:initialize_io_definition:634 - Initializing I/O Definition.
2025-01-09 14:59:10.457 | DEBUG    | roc_800.client:get_physical_io_definition:573 - Requesting Physical I/O Definition.
2025-01-09 14:59:10.457 | DEBUG    | roc_800.client:get_physical_io_definition:576 - Retrieving Logical I/O Numbers.
2025-01-09 14:59:10.459 | DEBUG    | roc_

[
  {
    "history_definition": {}
  },
  {
    "io_definition": {
      "0": {
        "physical_location": 0,
        "logical_number": 0,
        "point_type": 109,
        "point_tag_id": null
      },
      "1": {
        "physical_location": 1,
        "logical_number": 1,
        "point_type": 109,
        "point_tag_id": null
      },
      "2": {
        "physical_location": 2,
        "logical_number": 2,
        "point_type": 109,
        "point_tag_id": null
      },
      "3": {
        "physical_location": 3,
        "logical_number": 3,
        "point_type": 109,
        "point_tag_id": null
      },
      "4": {
        "physical_location": 4,
        "logical_number": 4,
        "point_type": 109,
        "point_tag_id": null
      },
      "5": {
        "physical_location": 5,
        "logical_number": 0,
        "point_type": 0,
        "point_tag_id": null
      },
      "6": {
        "physical_location": 6,
        "logical_number": 0,
        "point_type": 0,
  

### Script to generate py file

In [7]:
import json
import os
from typing import Dict, Any
import re
import csv

point_type_names_file_path = "C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\Parsed_ROC_Plus_files\\point_type_names.csv"
json_folder_path = "C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\Parsed_ROC_Plus_files\\textract_jsons"
dest_folder_path = "C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\rocpy\\roc_parameters"

def clean_string(str_val: str) -> str:
    clean_str = str_val.replace('"','').replace('\n','').strip()
    clean_str = clean_str.replace('Al ', 'AI ')
    return clean_str

def format_multiline_string(str_val: str) -> str:
    words: list[str] = str_val.split()
    lines= ['"""']
    current_line = []

    current_length = 0
    for word in words:
        if current_length + len(word) + len(current_line) > 160:
            lines.append(' '.join(current_line))
            current_line = [word]
            current_length = len(word)
        else:
            current_line.append(word)
            current_length += len(word)
    if current_line:
        lines.append(' '.join(current_line))
    lines.append('"""')

    # return '"""\n        ' + '        \n'.join(lines) + '\n        """'
    return '\n        '.join(lines)

def extract_param_number(param_obj: Dict) -> int:
    return int(param_obj['param_number'])

def extract_param_name(param_obj: Dict) -> str:
    return clean_string(param_obj['name'])

def extract_param_data_type(param_obj: Dict) -> str:
    data_type: str = clean_string(param_obj['data_type'])
    data_type = data_type.replace(' ','').upper()
    if data_type in ['U8', 'UNIT8']:
        data_type = 'UINT8'
    if data_type in ['U32', 'UNIT32']:
        data_type = 'UINT32'
    if data_type in ['FLT', 'FLOATFLOAT']:
        data_type = 'FLOAT'
    return data_type

def extract_access(param_obj: Dict) -> str:
    access: str = param_obj['access']
    return clean_string(access.upper())

def extract_range(param_obj: Dict) -> str:
    two_integers_pattern = r'^-?\d{1,3}(,\d{3})* -?\d{1,3}(,\d{3})*$'
    value_range: str = str(param_obj['range'])
    if value_range == '0 1':
        return '0-1'
    elif value_range == '0x00 0xFF':
        return '0x00-0xFF'
    elif value_range == '1 86,400':
        return '1-86400'
    elif value_range == '0 255':
        return '0-255'
    elif re.fullmatch(two_integers_pattern, value_range):
        first_int = value_range.split(' ')[0]
        second_int = value_range.split(' ')[1]
        return f'{first_int}-{second_int}'
    else:
        return value_range

def extract_description(param_obj: Dict) -> str:
    param_description: str = param_obj['desc']
    if param_description == '':
        param_description = param_obj['name']
    param_description = clean_string(param_description)
    return param_description

def extract_comment(param_obj: Dict) -> str:
    param_comment: str = param_obj['desc']
    if param_comment == '':
        param_comment = param_obj['name']
    param_comment = clean_string(param_comment)
    return format_multiline_string(param_comment)

def get_param_var(param_desc: str) -> str:
    # Remove basic stuff
    param_var: str = clean_string(param_desc)
    
    # Replace special characters & spaces with underscore
    param_var = re.sub(r'[^a-zA-Z0-9]', '_', param_var).replace(' ', '_')

    # If starts with number, add underscore
    try:
        start_int = int(param_var[0])
        param_var = '_' + param_var
    except:
        pass

    # Cast uppercase
    param_var = param_var.upper()

    # Remove any goofy multi-underscores
    param_var = param_var.replace('__', '_').replace('___', '_')

    # Normalize Point Tag ID
    if param_var == 'POINT_TAG_IDENTIFICATION':
        param_var = 'POINT_TAG_ID'

    return param_var

def get_point_type_name(input_point_type) -> Dict[str, str] | None:
    with open(point_type_names_file_path, mode='r', newline='', encoding='utf-8') as csv_file:
        reader = csv.DictReader(csv_file)
        for row in reader:
            if int(row['Point Type']) == input_point_type:
                point_type_var: str = row['Name']
                point_type_desc: str = row['Description']
                return {
                    'var': point_type_var,
                    'desc': point_type_desc
                }
        return None

all_code_list: list[str] = []

import_def: list[str] = []
import_def.append('from .parameter import BitDescriptor, Parameter, ParameterBinary')
import_def.append('from .point_type import PointType')
import_def.append('from roc_data_types import ParameterDataTypes as dt')
import_def.append('from enum import Enum')
import_def_code: str = '\n'.join(import_def)
all_code_list.append(import_def_code)

for file_name in os.listdir(json_folder_path):
    print(f'Processing {file_name}')

    # Get point type
    point_type: str = file_name.replace('point_type_','').replace('.json', '')
    point_type_int = int(point_type)
    
    # Get point type variable/description info
    point_type_dict = get_point_type_name(point_type_int)
    if point_type_dict is None:
        raise ValueError
    point_type_var = point_type_dict['var']
    point_type_desc = point_type_dict['desc']

    # Generate top-level point type definition code
    point_type_def: list[str] = []
    point_type_def.append(f'class {point_type_var}(PointType):')
    point_type_def.append(f'    """{point_type_desc}"""\n')
    point_type_def.append(f'    point_type_number: int = {int(point_type_int)}')
    point_type_def.append(f'    """Point Type ID"""\n')
    point_type_def.append(f"    point_type_desc: str = '{point_type_desc}'")
    point_type_def.append(f'    """Point Type Description"""\n')
    point_type_def.append('    class Parameters:')
    point_type_def.append(f'        class PointTypeParameter(Parameter):')
    point_type_def.append(f'            pass\n')
    point_type_def.append(f'        class PointTypeParameterBinary(ParameterBinary):')
    point_type_def.append(f'            pass\n')
    point_type_def_code: str = '\n'.join(point_type_def)

    # Get JSON
    json_file_path: str = os.path.join(json_folder_path, file_name)
    with open(json_file_path, 'r') as f:
        parameters: list[Dict] = json.load(f)

    # Sort parameters by parameter number
    parameters: list[Dict] = sorted(parameters, key=lambda x: int(x['param_number']))

    # Loop through parameters
    param_defs: list[str] = []
    for parameter in parameters:

        # Extract values
        param_number: int = extract_param_number(parameter)
        param_name: str = extract_param_name(parameter)
        param_desc: str = extract_description(parameter)
        param_var: str = get_param_var(param_name)
        data_type: str = extract_param_data_type(parameter)
        access: str = extract_access(parameter)
        value_range: str = extract_range(parameter)
        comment: str = extract_comment(parameter)

        # Filter out RESERVED/UNUSED parameters
        if param_var in ['RESERVED', 'UNUSED']:
            continue

        # Add data for binary types
        if data_type == 'BIN':
            param_type = 'PointTypeParameterBinary'
            bits_def = '            bits=['
            all_bit_def = [bits_def]
            for bit in range(8):
                if f'bit_{bit}' in parameter:
                    bit_data: dict = parameter[f'bit_{bit}']
                    bit_def: list[str] = []
                    bit_def.append(f'                BitDescriptor(')
                    bit_def.append(f'                    bit_number={bit},')
                    bit_def.append(f"                    bit_name='{bit_data['name']}',")
                    bit_def.append(f"                    bit_desc='{bit_data['desc']}'")
                    if bit == 7:
                        bit_def.append(f'                )')
                    else:
                        bit_def.append(f'                ),')
                    all_bit_def.append('\n'.join(bit_def))
            all_bit_def.append('            ]')
        else:
            param_type = 'PointTypeParameter'

        # Construct code string
        param_def: list[str] = []
        param_def.append(f'        {param_var} = {param_type}(')
        param_def.append(f'            parameter_number={param_number},')
        param_def.append(f"            parameter_name='{param_name}',")
        param_def.append(f"            parameter_desc='{param_desc}',")
        param_def.append(f'            data_type=dt.{data_type},')
        param_def.append(f"            access='{access}',")
        # param_def.append(f"            point_type_name='{point_type_var}',")
        if data_type == 'BIN':
            param_def.append(f"            value_range='{value_range}',")
            param_def.extend(all_bit_def)
        else:
            param_def.append(f"            value_range='{value_range}'")
        param_def.append(f'        )')
        param_def.append(f'        {comment}')
        param_def_code: str = '\n'.join(param_def)
        
        # Add to parameters code string
        param_defs.append(param_def_code)

    # Generate final code output for point type
    all_params_code = '\n\n'.join(param_defs)
    point_type_code: str = f'{point_type_def_code}\n\n{all_params_code}'

    all_code_list.append(point_type_code)

final_code: str = '\n\n\n\n\n\n'.join(all_code_list)

dest_file_path = os.path.join(dest_folder_path, f'point_type_definitions.py')
with open(dest_file_path, 'w') as f:
    f.write(final_code)

Processing point_type_100.json
Processing point_type_101.json
Processing point_type_102.json
Processing point_type_103.json
Processing point_type_104.json
Processing point_type_105.json
Processing point_type_106.json
Processing point_type_107.json
Processing point_type_108.json
Processing point_type_109.json
Processing point_type_110.json
Processing point_type_111.json
Processing point_type_112.json
Processing point_type_113.json
Processing point_type_114.json
Processing point_type_115.json
Processing point_type_116.json
Processing point_type_117.json
Processing point_type_118.json
Processing point_type_119.json
Processing point_type_120.json
Processing point_type_121.json
Processing point_type_122.json
Processing point_type_123.json
Processing point_type_124.json
Processing point_type_125.json
Processing point_type_126.json
Processing point_type_127.json
Processing point_type_128.json
Processing point_type_129.json
Processing point_type_130.json
Processing point_type_131.json
Processi

### Script to split PDF into multiple PDFs

In [3]:
import pandas as pd
from PyPDF2 import PdfReader, PdfWriter

split_map_csv = 'C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\Parsed_ROC_Plus_files\\point_type_pdfs\\point_type_pdf_map.csv'
df: pd.DataFrame = pd.read_csv(split_map_csv)

input_pdf = "C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\roc-plus-protocol-specification-manual-en-6851618.pdf"
reader = PdfReader(input_pdf)

for _, row in df.iterrows():
    start_page = row['Start Page'] - 1
    end_page = row['End Page']
    output_filename = row['Output Filename']

    writer = PdfWriter()
    for page_num in range(start_page, end_page):
        writer.add_page(reader.pages[page_num])
    
    with open(output_filename, 'wb') as output_pdf:
        writer.write(output_pdf)

### Script to read in ALL Textract results and cast to JSON

In [31]:
import os
import zipfile
import pandas as pd
import json

textract_result_dir = "C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\Parsed_ROC_Plus_files\\textract_results"
for file_name in os.listdir(textract_result_dir):
    point_type = file_name.replace('point_type_','').replace('.zip', '')
    print(f'Working on point type {point_type}')
    zip_path = f"C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\Parsed_ROC_Plus_files\\textract_results\\point_type_{point_type}.zip"
    output_path = f"C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\Parsed_ROC_Plus_files\\textract_jsons\\point_type_{point_type}.json"

    temp_dir = f'temp_csv_folder_{point_type}'
    os.makedirs(temp_dir, exist_ok=True)


    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)

    all_data = []

    for file_name in os.listdir(temp_dir):
        if file_name.endswith('.csv'):
            file_path: str = os.path.join(temp_dir, file_name)
            try:
                df: pd.DataFrame = pd.read_csv(file_path)
            except:
                print('Failed to get data frame. Skipping.')
                continue
            for _, row in df.iterrows():
                if "'Param#" not in row.keys() and "'Param #" not in row.keys() and "'Param" not in row.keys():
                    continue
                try:
                    param_number = row["'Param#"]
                except KeyError:
                    try:
                        param_number = row["'Param #"]
                    except KeyError:
                        param_number = row["'Param"]
                if not isinstance(param_number, str):
                        break
                param_number = str(param_number).replace("'", '')
                param_name = str(row["'Name"]).replace("'",'')
                access = str(row["'Access"]).replace("'",'')
                update_type = str(row["'System or User Update"]).replace("'",'')
                data_type = str(row["'Data Type"]).replace("'",'')
                length = str(row["'Length"]).replace("'",'')
                range_ = str(row["'Range"]).replace("'",'')
                default = str(row["'Default"]).replace("'",'')
                version = str(row["'Ver"]).replace("'",'')
                desc = str(row["'Description of functionality and meaning of values"]).replace("'",'')
                row_dict = {
                    'param_number': param_number,
                    'name': param_name,
                    'access': access,
                    'update_type': update_type,
                    'data_type': data_type,
                    'length': length,
                    'range': range_,
                    'default': default,
                    'version': version,
                    'desc': desc
                }
                all_data.append(row_dict)
    # print(all_data)
    # json_str: str = json.dumps(all_data)
    # print(json_str)

    with open(output_path, 'w') as json_file:
        json.dump(all_data, json_file, indent=4)

    # for file_name in os.listdir(temp_dir):
    #     os.remove(os.path.join(temp_dir, file_name))
    # os.rmdir(temp_dir)

Working on point type 100
Working on point type 101
Working on point type 102
Working on point type 103
Working on point type 104
Working on point type 105
Working on point type 106
Working on point type 107
Working on point type 108
Working on point type 109
Working on point type 110
Working on point type 111
Working on point type 112
Working on point type 113
Working on point type 114
Working on point type 115
Working on point type 115
Working on point type 116
Working on point type 117
Working on point type 118
Working on point type 119
Working on point type 120
Working on point type 121
Working on point type 122
Working on point type 123
Working on point type 124
Working on point type 125
Working on point type 126
Working on point type 127
Working on point type 128
Working on point type 129
Working on point type 130
Working on point type 131
Working on point type 132
Working on point type 133
Working on point type 134
Working on point type 135
Working on point type 136
Working on p

### Script to find issues in JSONs

In [119]:
import json
import os

# Figure out bitwise stuff

json_folder = 'C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\Parsed_ROC_Plus_files\\textract_jsons'

problem_dict = {}
for file_name in os.listdir(json_folder):
    file_path = os.path.join(json_folder, file_name)
    with open(file_path, 'r') as f:
        obj: list[dict] = json.load(f)
        for row in obj:
            param_number = row['param_number']
            if 'Series' in param_number:
                number = param_number.split('(')[0].strip()
                series = param_number.split('(')[1].replace('Series ','').replace(')','')
                param_number = number + '_SERIES_' + series
            if 'HART' in param_number:
                number = param_number.split('(')[0].strip()
                hart = param_number.split('(')[1].replace('HART ','').replace(')','')
                param_number = number + '_HART_' + hart
            try:
                intparam = int(param_number)
            except:
                if file_name not in problem_dict:
                    problem_dict[file_name] = []
                problem_dict[file_name].append(param_number)
print(problem_dict)
print(len(problem_dict))

{}
0


### Script to fix bit addresses

In [118]:
import json
import os

json_folder = 'C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\Parsed_ROC_Plus_files\\textract_jsons'

problem_dict = {}
for file_name in os.listdir(json_folder):
    file_path = os.path.join(json_folder, file_name)
    with open(file_path, 'r') as f:
        obj: list[dict] = json.load(f)
        for row in obj:
            param_number = row['param_number']
            if 'Series' in param_number:
                number = param_number.split('(')[0].strip()
                series = param_number.split('(')[1].replace('Series ','').replace(')','')
                param_number = number + '_SERIES_' + series
            if 'HART' in param_number:
                number = param_number.split('(')[0].strip()
                hart = param_number.split('(')[1].replace('HART ','').replace(')','')
                param_number = number + '_HART_' + hart
            try:
                intparam = int(param_number)
            except:
                if file_name not in problem_dict:
                    problem_dict[file_name] = []
                problem_dict[file_name].append(param_number)
print(problem_dict)
print(len(problem_dict))

for json_file, bit_addresses in problem_dict.items():
    print(f'Processing {json_file}')
    file_path = os.path.join(json_folder, json_file)
    with open(file_path, 'r') as f:
        obj: list[dict] = json.load(f)
    bit_dict = {}
    for bit_address in bit_addresses:
        param_int = bit_address.split('.')[0]
        bit_int = bit_address.split('.')[1]
        if param_int not in bit_dict:
            bit_dict[param_int] = []
        bit_dict[param_int].append(bit_int)
    print(bit_dict)
    for root_address, bit_addresses in bit_dict.items():
        print(f'Looking for parameter {root_address} in {json_file}')
        root_param = None
        for row in obj:
            if row['param_number'] == str(root_address):
                root_param = row
                print('Root parameter found')
                break
        if root_param is None:
            print('Error!!!!')
            continue
        for bit_address in bit_addresses:
            bit_param_number = f'{root_address}.{bit_address}'
            bit_param = None
            for row in obj:
                if row['param_number'] == str(bit_param_number):
                    bit_param = row
                    break
            if bit_param is None:
                print('Error!!!')
                continue
            bit_name = bit_param['name']
            bit_desc = bit_param['desc']
            root_param[f'bit_{bit_address}'] = {
                'name': bit_name,
                'desc': bit_desc
            }
            bit_index = None
            for i, row in enumerate(obj):
                if row['param_number'] == str(bit_param_number):
                    bit_index = i
            if bit_index is None:
                print('Error!!!')
                continue
            obj.pop(bit_index)
    print(obj)
    with open(file_path, 'w') as f:
        json.dump(obj, f, indent=4)

{'point_type_101.json': ['12.0', '12.1', '12.2', '12.3', '12.4', '12.5', '12.6', '12.7'], 'point_type_102.json': ['6.0', '6.1', '6.2', '6.3', '6.4', '6.5', '6.6', '6.7'], 'point_type_103.json': ['32.0', '32.1', '32.2', '32.3', '32.4', '32.5', '32.6', '32.7'], 'point_type_104.json': ['6.0', '6.1', '6.2', '6.3', '6.4', '6.5', '6.6', '6.7'], 'point_type_105.json': ['23.0', '23.1', '23.2', '23.3', '23.4', '23.5', '23.6', '23.7'], 'point_type_106.json': ['33.0', '33.1', '33.2', '33.3', '33.4', '33.5', '33.6', '33.7'], 'point_type_107.json': ['19.0', '19.1', '19.2', '19.3', '19.4', '19.5', '19.6', '19.7'], 'point_type_108.json': ['10.0', '10.1', '10.2', '10.3', '10.4', '10.5', '10.6', '10.7', '11.0', '11.1', '11.2', '11.3', '11.4', '11.5', '11.6', '11.7', '28.0', '28.1', '28.2', '28.3', '28.4', '28.5', '28.6', '28.7', '43.0', '43.1', '43.2', '43.3', '43.4', '43.5', '43.6', '43.7', '58.0', '58.1', '58.2', '58.3', '58.4', '58.5', '58.6', '58.7'], 'point_type_109.json': ['23.0', '23.1', '23.2',

### Script to fix point type 141

In [87]:
import json
import os

json_path = "C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\Parsed_ROC_Plus_files\\textract_jsons\\point_type_141.json"

series_keys = []
with open(json_path, 'r') as f:
    obj: dict = json.load(f)
    for row in obj:
        if 'Series' in row['param_number']:
            param_int = row['param_number'].split('(')[0].strip()
            row['param_number'] = param_int
print(obj)

with open(json_path, 'w') as f:
    json.dump(obj, f, indent=4)

[{'param_number': '60', 'name': 'API Reverse Pulse Counts Pair 1', 'access': 'R/O', 'update_type': 'System', 'data_type': 'UINT32', 'length': '4', 'range': '0 16,000,000', 'default': '0', 'version': '3.00', 'desc': 'Indicates the accumulated number of reverse pulses through the API level checks for pulse pair 1. The system updates this value only when the API Level Check Pair 1 (parameter #1) is set to either Level B or C.'}, {'param_number': '61', 'name': 'API Reverse Pulse Counts Pair 2', 'access': 'R/O', 'update_type': 'System', 'data_type': 'UINT32', 'length': '4', 'range': '0 -> 16,000,000', 'default': '0', 'version': '3.00', 'desc': 'Indicates the accumulated number of reverse pulses through the API level checks for pulse pair 2. The system updates this value only when the API Level Check Pair 2 (parameter #2) is set to either Level B or C.'}, {'param_number': '62', 'name': 'Pulse Input 1 Tag', 'access': 'R/W', 'update_type': 'User', 'data_type': 'AC', 'length': '20', 'range': '0

### Script to fix point type 121

In [48]:
import json
import os

json_path = "C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\Parsed_ROC_Plus_files\\textract_jsons\\point_type_121.json"

series_keys = []
with open(json_path, 'r') as f:
    obj: list[Dict] = json.load(f)

obj: list[Dict] = sorted(obj, key=lambda x: int(x['param_number']))
rtu = 0
for row in obj:
    if row['param_number'] != '0':
        if 'RTU' in row['name']:
            rtu = row['name'].strip().replace('RTU ','').replace(' Address', '')
        else:
            row['name'] = f'RTU {rtu} {row['name']}'

print(obj)

with open(json_path, 'w') as f:
    json.dump(obj, f, indent=4)

[{'param_number': '0', 'name': 'Tag ID', 'access': 'R/W', 'update_type': 'User', 'data_type': 'AC', 'length': '10', 'range': '0x20 0x7E for each byte', 'default': '"MastTbl #', 'version': '1.10', 'desc': 'String that describes the instance of the Master table.'}, {'param_number': '1', 'name': 'RTU 1 Address', 'access': 'R/W', 'update_type': 'User', 'data_type': 'UINT8', 'length': '1', 'range': '0 255', 'default': '0', 'version': '1.10', 'desc': 'Contains RTU 1 Address the Modbus Query is destined for'}, {'param_number': '2', 'name': 'RTU 1 Function Code Number', 'access': 'R/W', 'update_type': 'User', 'data_type': 'UINT8', 'length': '1', 'range': '0 6, 15, 16', 'default': '0', 'version': '1.10', 'desc': 'Specifies the Modbus Function Code to be sent to the slave device on RTU 1. Note: See Modbus Function Codes list at end of table.'}, {'param_number': '3', 'name': 'RTU 1 Slave Register Number', 'access': 'R/W', 'update_type': 'User', 'data_type': 'UINT16', 'length': '2', 'range': '0 65

### Script to fix point type 119

In [54]:
import json
import os

json_path = "C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\Parsed_ROC_Plus_files\\textract_jsons\\point_type_119.json"

series_keys = []
with open(json_path, 'r') as f:
    obj: list[Dict] = json.load(f)

obj: list[Dict] = sorted(obj, key=lambda x: int(x['param_number']))
history_range = 0
for row in obj:
    if int(row['param_number']) > 2:
        if '#' in row['name']:
            history_range = row['name'].strip().split('#')[1]
        else:
            row['name'] = f'{row['name']} #{history_range}'

print(obj)

with open(json_path, 'w') as f:
    json.dump(obj, f, indent=4)

[{'param_number': '0', 'name': 'Event/Alarm Register', 'access': 'R/W', 'update_type': 'User', 'data_type': 'UINT16', 'length': '2', 'range': '0 65535', 'default': '32', 'version': '1.00', 'desc': 'Contains a unique register number that indicates the request is for Events and Alarm records.'}, {'param_number': '1', 'name': 'Current Date Register', 'access': 'R/W', 'update_type': 'User', 'data_type': 'UINT16', 'length': '2', 'range': '0 65535', 'default': '7046', 'version': '1.00', 'desc': 'Contains a unique register that allows a Modbus read/write command to access the current date in MMDDYY format'}, {'param_number': '2', 'name': 'Current Time Register', 'access': 'R/W', 'update_type': 'User', 'data_type': 'UINT16', 'length': '2', 'range': '0 65535', 'default': '7047', 'version': '1.00', 'desc': 'Contains a unique register that allows a Modbus read/write command to access the current time in HHMMSS format'}, {'param_number': '3', 'name': 'Periodic History Register #1', 'access': 'R/W'

### Script to fix point type 137

In [61]:
import json
import os

json_path = "C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\Parsed_ROC_Plus_files\\textract_jsons\\point_type_137.json"

series_keys = []
with open(json_path, 'r') as f:
    obj: list[Dict] = json.load(f)

obj: list[Dict] = sorted(obj, key=lambda x: int(x['param_number']))
modbus_master_table = 0
server = 0
for row in obj:
    if int(row['param_number']) > 17:
        if 'Modbus Master TCP' in row['name']:
            modbus_master_table += 1
            server = 0
        elif 'IP Address' in row['name']:
            server += 1
        
        if 'Modbus Master TCP' in row['name']:
            row['name'] = f'Table {modbus_master_table} {row['name']}'
        elif 'Test' in row['name']:
            row['name'] = row['name']
        else:
            row['name'] = f'Table {modbus_master_table} Server {server} {row['name']}'

print(obj)

with open(json_path, 'w') as f:
    json.dump(obj, f, indent=4)

[{'param_number': '0', 'name': 'MAC Address', 'access': 'R/O', 'update_type': 'System', 'data_type': 'AC12', 'length': '12', 'range': 'N/A', 'default': 'Varies', 'version': '1.00', 'desc': 'Unique MAC address set by the factory.'}, {'param_number': '1', 'name': 'IP Address', 'access': 'R/W', 'update_type': 'User', 'data_type': 'AC20', 'length': '20', 'range': 'See note in description', 'default': '10.0.0.2', 'version': '1.00', 'desc': 'IP address for the ROC800. Note: These values must be in the format XXX.XXX.XXX.XXX (such as 10.0.0.1). The value 255.255.255.255 is invalid.'}, {'param_number': '2', 'name': 'Subnet Mask', 'access': 'R/W', 'update_type': 'User', 'data_type': 'AC20', 'length': '20', 'range': 'See note in description', 'default': '255.255.255. 0', 'version': '1.00', 'desc': 'Subnet mask for the ROC800. Note: These values must be in the format XXX.XXX.XXX.XXX (such as 10.0.0.1). The value 255.255.255.255 is invalid'}, {'param_number': '3', 'name': 'Gateway Address', 'acces

### Script to fix point type 145

In [16]:
import json
import os
import math

json_path = "C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\Parsed_ROC_Plus_files\\textract_jsons\\point_type_145.json"

with open(json_path, 'r') as f:
    obj: dict = json.load(f)
    for row in obj:
        if int(row['param_number']) == 2:
            parameter_2 = row
            row['name'] = 'Point Description 1'
        elif int(row['param_number']) == 3:
            parameter_3 = row
            row['name'] = 'Point to Log 1'

for i in range(4, 201, 2):
    desc_index = str(i)
    log_index = str(i + 1)
    pair = int(i / 2)
    new_parameter_2 = {k: v for k, v in parameter_2.items()}
    new_parameter_2['param_number'] = desc_index
    new_parameter_2['name'] = f'Point Description {pair}'
    new_parameter_3 = {k: v for k, v in parameter_3.items()}
    new_parameter_3['param_number'] = log_index
    new_parameter_3['name'] = f'Point to Log {pair}'
    obj.append(new_parameter_2)
    obj.append(new_parameter_3)
print(obj)
with open(json_path, 'w') as f:
    json.dump(obj, f, indent=4)

[{'param_number': '0', 'name': 'Write Trigger', 'access': 'R/W', 'update_type': 'User', 'data_type': 'UINT8', 'length': '1', 'range': '0-1', 'default': 'No Tag', 'version': '3.60', 'desc': 'Indicates the write trigger for a transaction. Valid values are 0 (Idle) and 1 (write transaction).'}, {'param_number': '1', 'name': 'Transaction Description', 'access': 'R/W', 'update_type': 'User', 'data_type': 'AC', 'length': '10', 'range': '0x20 0x7E for each byte', 'default': '', 'version': '3.60', 'desc': 'Transaction description (can be changed without changing transactional history)'}, {'param_number': '2', 'name': 'Point Description 1', 'access': 'R/W', 'update_type': 'User', 'data_type': 'AC', 'length': '10', 'range': '0x20 0x7E for each byte', 'default': '', 'version': '3.60', 'desc': 'Point description (can be changed without changing transactional history)'}, {'param_number': '3', 'name': 'Point to Log 1', 'access': 'R/W', 'update_type': 'User', 'data_type': 'TLP', 'length': '3', 'range

### Script to fix point type 85

In [93]:
import json
import os

json_path = "C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\Parsed_ROC_Plus_files\\textract_jsons\\point_type_85.json"

series_keys = []
with open(json_path, 'r') as f:
    obj: dict = json.load(f)
    param_numbers = [row['param_number'] for row in obj]
    for param_number in param_numbers:
        if 'HART' in param_number:
            series_keys.append(param_number)
print(series_keys)

series_dict = {}
for series_key in series_keys:
    param_int = series_key.split('(')[0].strip()
    if param_int not in series_dict:
        series_dict[param_int] = []
    series_dict[param_int].append(series_key)
print(series_dict)

def get_row_for_param_num(param_num):
    for row in obj:
        if row['param_number'] == param_num:
            return row

def get_index_for_param_num(param_num):
    for i, row in enumerate(obj):
        if row['param_number'] == param_num:
            return i

for param_int, series_keys in series_dict.items():
    print(f'Processing param {param_int}')
    row_series_1 = get_row_for_param_num(series_keys[0])
    row_series_2 = get_row_for_param_num(series_keys[1])
    combined_row = {k: v for k, v in row_series_1.items()}
    combined_range = f'HART 1: {row_series_1['range']}; HART 2: {row_series_2['range']}'
    combined_version = f'HART 1: {row_series_1['version']}; HART 2: {row_series_2['version']}'
    combined_desc = f'HART 1: {row_series_1['desc']}; HART 2: {row_series_2['desc']}'
    combined_row['param_number'] = param_int
    combined_row['range'] = combined_range
    combined_row['version'] = combined_version
    combined_row['desc'] = combined_desc
    obj.pop(get_index_for_param_num(series_keys[0]))
    obj.pop(get_index_for_param_num(series_keys[1]))
    obj.append(combined_row)
print(obj)

with open(json_path, 'w') as f:
    json.dump(obj, f, indent=4)

['115 (HART 1)', '115 (HART 2)', '124 (HART 1)', '124 (HART 2)', '162 (HART 1)', '162 (HART 2)', '171 (HART 1)', '171 (HART 2)', '209 (HART 1)', '209 (HART 2)', '218 (HART 1)', '218 (HART 2)', '0 (HART 1)', '0 (HART 2)', '1 (HART 1)', '1 (HART 2)', '2 (HART 1)', '2 (HART 2)', '4 (HART 1)', '4 (HART 2)', '7 (HART 1)', '7 (HART 2)', '21 (HART 1)', '21 (HART 2)', '30 (HART 1)', '30 (HART 2)', '68 (HART 1)', '68 (HART 2)', '77 (HART 1)', '77 (HART 2)']
{'115': ['115 (HART 1)', '115 (HART 2)'], '124': ['124 (HART 1)', '124 (HART 2)'], '162': ['162 (HART 1)', '162 (HART 2)'], '171': ['171 (HART 1)', '171 (HART 2)'], '209': ['209 (HART 1)', '209 (HART 2)'], '218': ['218 (HART 1)', '218 (HART 2)'], '0': ['0 (HART 1)', '0 (HART 2)'], '1': ['1 (HART 1)', '1 (HART 2)'], '2': ['2 (HART 1)', '2 (HART 2)'], '4': ['4 (HART 1)', '4 (HART 2)'], '7': ['7 (HART 1)', '7 (HART 2)'], '21': ['21 (HART 1)', '21 (HART 2)'], '30': ['30 (HART 1)', '30 (HART 2)'], '68': ['68 (HART 1)', '68 (HART 2)'], '77': ['77 

### Script to fix point type 118

In [84]:
import json
import os

json_path = "C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\Parsed_ROC_Plus_files\\textract_jsons\\point_type_118.json"

series_keys = []
with open(json_path, 'r') as f:
    obj: dict = json.load(f)
    param_numbers = [row['param_number'] for row in obj]
    for param_number in param_numbers:
        if 'Series' in param_number:
            series_keys.append(param_number)
print(series_keys)

series_dict = {}
for series_key in series_keys:
    param_int = series_key.split('(')[0].strip()
    if param_int not in series_dict:
        series_dict[param_int] = []
    series_dict[param_int].append(series_key)
print(series_dict)

def get_row_for_param_num(param_num):
    for row in obj:
        if row['param_number'] == param_num:
            return row

def get_index_for_param_num(param_num):
    for i, row in enumerate(obj):
        if row['param_number'] == param_num:
            return i

for param_int, series_keys in series_dict.items():
    print(f'Processing param {param_int}')
    row_series_1 = get_row_for_param_num(series_keys[0])
    row_series_2 = get_row_for_param_num(series_keys[1])
    combined_row = {k: v for k, v in row_series_1.items()}
    combined_range = f'Series 1: {row_series_1['range']}; Series 2: {row_series_2['range']}'
    combined_version = f'Series 1: {row_series_1['version']}; Series 2: {row_series_2['version']}'
    combined_row['param_number'] = param_int
    combined_row['range'] = combined_range
    combined_row['version'] = combined_version
    obj.pop(get_index_for_param_num(series_keys[0]))
    obj.pop(get_index_for_param_num(series_keys[1]))
    obj.append(combined_row)
print(obj)

with open(json_path, 'w') as f:
    json.dump(obj, f, indent=4)

[]
{}
[{'param_number': '0', 'name': 'Tag ID', 'access': 'R/W', 'update_type': 'User', 'data_type': 'AC', 'length': '10', 'range': '0x20 0x7E for each byte', 'default': 'Reg Map #', 'version': '1.00', 'desc': 'String that describes the instance of the mapping table.'}, {'param_number': '1', 'name': 'Start Register #1', 'access': 'R/W', 'update_type': 'User', 'data_type': 'UINT16', 'length': '2', 'range': '0 65535', 'default': '0', 'version': '1.00', 'desc': 'The starting register number for the first range of Modbus registers that map to ROC Plus Protocol TLP(s).'}, {'param_number': '2', 'name': 'End Register #1', 'access': 'R/W', 'update_type': 'User', 'data_type': 'UINT16', 'length': '2', 'range': '0 65535', 'default': '0', 'version': '1.00', 'desc': 'The ending register number for the first range of Modbus registers that map to ROC Plus Protocol TLP(s).'}, {'param_number': '3', 'name': 'ROC Parameter(s) (Reg Range 1)', 'access': 'R/W', 'update_type': 'User', 'data_type': 'TLP', 'len

### Script to re-generate select JSON from Textual results

In [57]:
import os
import zipfile
import pandas as pd
import json

re_imports = [123, 140, 85, 95]

textract_result_dir = "C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\Parsed_ROC_Plus_files\\textract_results"
for file_name in os.listdir(textract_result_dir):
    point_type = file_name.replace('point_type_','').replace('.zip', '')
    zip_path = f"C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\Parsed_ROC_Plus_files\\textract_results\\point_type_{point_type}.zip"
    output_path = f"C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\Parsed_ROC_Plus_files\\textract_jsons\\point_type_{point_type}.json"

    if int(point_type) not in re_imports:
        continue

    print(f'Working on point type {point_type}')

    temp_dir = f'temp_csv_folder_{point_type}'
    os.makedirs(temp_dir, exist_ok=True)


    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)

    all_data = []

    for file_name in os.listdir(temp_dir):
        if file_name.endswith('.csv'):
            file_path: str = os.path.join(temp_dir, file_name)
            try:
                df: pd.DataFrame = pd.read_csv(file_path)
            except:
                print('Failed to get data frame. Skipping.')
                continue
            for _, row in df.iterrows():
                if "'Param#" not in row.keys() and "'Param #" not in row.keys() and "'Param" not in row.keys():
                    continue
                try:
                    param_number = row["'Param#"]
                except KeyError:
                    try:
                        param_number = row["'Param #"]
                    except KeyError:
                        param_number = row["'Param"]
                if not isinstance(param_number, str):
                        break
                param_number = str(param_number).replace("'", '')
                param_name = str(row["'Name"]).replace("'",'')
                access = str(row["'Access"]).replace("'",'')
                update_type = str(row["'System or User Update"]).replace("'",'')
                data_type = str(row["'Data Type"]).replace("'",'')
                length = str(row["'Length"]).replace("'",'')
                range_ = str(row["'Range"]).replace("'",'')
                default = str(row["'Default"]).replace("'",'')
                version = str(row["'Ver"]).replace("'",'')
                desc = str(row["'Description of functionality and meaning of values"]).replace("'",'')
                row_dict = {
                    'param_number': param_number,
                    'name': param_name,
                    'access': access,
                    'update_type': update_type,
                    'data_type': data_type,
                    'length': length,
                    'range': range_,
                    'default': default,
                    'version': version,
                    'desc': desc
                }
                all_data.append(row_dict)
    # print(all_data)
    # json_str: str = json.dumps(all_data)
    # print(json_str)

    with open(output_path, 'w') as json_file:
        json.dump(all_data, json_file, indent=4)

    # for file_name in os.listdir(temp_dir):
    #     os.remove(os.path.join(temp_dir, file_name))
    # os.rmdir(temp_dir)

Working on point type 123
Working on point type 140
Working on point type 85
Working on point type 95


In [ ]:
import json

file_path = 'C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\rocpy\\roc_docs\\point_type_103.json'

with open(file_path, 'r') as f:
    data = json.load(f)

class_type = 'AI'
point_type = 103
point_type_desc = 'Analog Input'

code_gen_string = 'from roc_parameters.base import PointType, TLPParameter\n'
code_gen_string += 'import roc_data_types as dt\n\n'
code_gen_string += f'class {class_type}(PointType):\n\n'
code_gen_string += f'    point_type = {point_type}\n\n'
code_gen_string += f"    point_type_desc = '{point_type_desc}'\n\n"

for param in data:
    parameter_number = param['Param#']
    try:
        parameter_number = int(parameter_number)
    except:
        continue
    parameter_name: str = param['Name']
    access = param['Access']
    data_type_string = param['Data Type']
    length = param['Length']
    try:
        length = int(length)
    except:
        continue
    value_range = param['Range']
    parameter_desc = param['Description']

    parameter_title = parameter_name.upper().replace(' ', '_').replace('#', '_').replace('-', '_').replace('/','_').replace('(','_').replace(')','')

    class_string = f'    {parameter_title} = TLPParameter(\n'
    class_string += f'        parameter_number={parameter_number},\n'
    class_string += f"        parameter_desc='{parameter_name}',\n"
    class_string += f'        data_type=dt.{data_type_string},\n'
    class_string += f"        access='{access}',\n"
    class_string += f"        value_range='{value_range}',\n"
    class_string += f'        point_type=point_type,\n'
    class_string += f'        point_type_desc=point_type_desc\n'
    class_string += f'    )\n'
    class_string += f'    """{parameter_desc}"""\n\n'

    code_gen_string += class_string

print(code_gen_string)

dest_file_path = 'C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\rocpy\\roc_parameters\\ai.py'

with open(dest_file_path, 'w') as f:
    f.write(code_gen_string)

In [ ]:
import re
import json


def find_headers(file_path):
    with open(file_path, 'r') as file:
        markdown_context = file.read()
    
    pattern = re.compile(r'^#{3,4}\s*.*json.*point.*type.*', re.IGNORECASE | re.MULTILINE)

    matches = pattern.findall(markdown_context)

    return matches

def get_blocks(file_path, headers):

    with open(file_path, 'r') as file:
        markdown_context = file.read()

    json_blocks = {}

    for header in headers:
        escaped_header = re.escape(header)
        json_block_pattern = re.compile(
            f'{escaped_header}\\r?\\n```json\\r?\\n(.*?)```', 
            re.DOTALL | re.IGNORECASE
        )
        matches = json_block_pattern.findall(markdown_context)
        parsed_blocks = []
        for match in matches:
            try:
                parsed_json = json.loads(match.strip())
                parsed_blocks.append(parsed_json)
            except json.JSONDecodeError as e:
                print(f'Error parsing json for header {header}: {e}')
        if parsed_blocks:
            json_blocks[header] = parsed_blocks
    return json_blocks

# Function to extract and combine JSON blocks by header
def extract_and_combine_json(file_path):
    with open(file_path, 'r') as file:
        markdown_content = file.read()

    # Regex pattern to match headers and their JSON blocks
    pattern = r"### JSON Format \((Point Type \d+[,:] [\w\s_]+)\)"

    # Find all matches (header and JSON block)
    matches = re.findall(pattern, markdown_content, re.DOTALL)

    combined_data = {}

    for header, json_data in matches:
        try:
            # Parse the JSON block
            parsed_json = json.loads(json_data)

            # Combine data under the same header
            if header not in combined_data:
                combined_data[header] = []

            if isinstance(parsed_json, list):
                combined_data[header].extend(parsed_json)
            else:
                combined_data[header].append(parsed_json)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for header '{header}': {e}")

    return combined_data

# Save combined JSON data to a file
def save_combined_json(output_path, combined_data):
    with open(output_path, 'w') as file:
        json.dump(combined_data, file, indent=4)

# Example usage
# file_path = 'markdown_with_json.md'  # Path to your markdown file
# output_path = 'combined_data.json'  # Path to save the combined JSON

# combined_data = extract_and_combine_json(file_path)
# save_combined_json(output_path, combined_data)

# print(f"Combined JSON data saved to {output_path}")

file_path = 'C:\\Users\\SHillis\\OneDrive - Coterra Energy\\Documents\\rocpy\\roc_docs\\full_output.md'
headers = find_headers(file_path=file_path)
json_blocks = get_blocks(file_path=file_path, headers=headers)
print(json_blocks.keys())

possible_keys = [str(i) for i in range(300)]

combined_blocks = {}
for k, v in json_blocks.items():
    for key in possible_keys:
        if key in k:
            if key in combined_blocks:
                if isinstance(v, list):
                    combined_blocks[key].extend(v)
                else:
                    raise TypeError(f'type{v}')
            else:
                if isinstance(v, list):
                    combined_blocks[key] = v
                else:
                    raise TypeError(f'type{v}')

keys = []
for list in combined_blocks['103']:
    for block in list:
        if isinstance(block, dict):
            if 'Param#' in block.keys():
                keys.append(block['Param#'])

print(keys)